In [8]:
import keras
import numpy as np
from keras.datasets import cifar10
(x_train,y_train),(x_test,y_test)=cifar10.load_data()
print(y_train.shape)
print(y_test.shape)
y_train=y_train.flatten()#展开
y_test=y_test.flatten()
print(y_train.shape)
print(y_test.shape)
print(x_train.shape)

(50000, 1)
(10000, 1)
(50000,)
(10000,)
(50000, 32, 32, 3)


In [9]:
#缩放数据
x_train=x_train.astype('float32')/255
x_test=x_test.astype('float32')/255
from keras.utils import np_utils
from keras.utils import to_categorical
#消除重复  即得到有多少种类
num_classes=len(np.unique(y_train))
#编码
y_train=to_categorical(y_train,num_classes)
y_test =to_categorical(y_test, num_classes)

In [10]:
#构造神经网络
from keras.models import Sequential
from keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense,Dropout
model=Sequential()
model.add(Convolution2D(filters=16,kernel_size=2,padding='same',activation='relu',input_shape=(32,32,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Convolution2D(filters=32,kernel_size=2,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Convolution2D(filters=64,kernel_size=2,padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.1))
#展开为向量放入MLP中
model.add(Flatten())
#全连接层
model.add(Dense(500,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10,activation='softmax'))

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 32, 32, 16)        208       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 16, 32)        2080      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 8, 8, 64)          8256      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 4, 4, 64)          0         
__________

In [11]:
#交叉熵
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [15]:
from keras.callbacks import ModelCheckpoint
checkpointer=ModelCheckpoint(filepath='cifar10.model.best.hdf5',verbose=1,save_best_only=True)
hist=model.fit(x_train,y_train,batch_size=128,epochs=10,validation_split=0.1,callbacks=[checkpointer],verbose=1,shuffle=True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/10
45000/45000 [==============================] - 53s 1ms/step - loss: 0.8839 - acc: 0.7031 - val_loss: 0.9398 - val_acc: 0.6816

Epoch 00001: val_loss improved from inf to 0.93980, saving model to cifar10.model.best.hdf5
Epoch 2/10
45000/45000 [==============================] - 52s 1ms/step - loss: 0.8195 - acc: 0.7212 - val_loss: 0.9414 - val_acc: 0.6800

Epoch 00002: val_loss did not improve from 0.93980
Epoch 3/10
45000/45000 [==============================] - 45s 994us/step - loss: 0.7875 - acc: 0.7300 - val_loss: 0.8571 - val_acc: 0.7210

Epoch 00003: val_loss improved from 0.93980 to 0.85714, saving model to cifar10.model.best.hdf5
Epoch 4/10
45000/45000 [==============================] - 43s 966us/step - loss: 0.7543 - acc: 0.7406 - val_loss: 1.0566 - val_acc: 0.6840

Epoch 00004: val_loss did not improve from 0.85714
Epoch 5/10
45000/45000 [==============================] - 49s 1ms/step - loss: 0.7325 - acc: 0.7486 - val

In [16]:
16

16

In [17]:
#提取训练好的权重
model.load_weights('cifar10.model.best.hdf5')
score=model.evaluate(x_test,y_test,verbose=1)
score[1]

10000/10000 [==============================] - 5s 473us/step


0.7229